In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
columns = ['플랜트', '플랜트명', '매장코드', '매장', '상품코드', '상품명', '차수', '주문일자', '운송장번호',
           '운송장출력여부', '주문취소여부', '주문번호', '받는이', '우편번호', '주소1', '주소2', '전화번호1',
           '전화번호2', '배송기재사항', '판매가', '합포여부', '예정수량', '검수수량', '출력일시', '출력자', '몰구분', 'DAS여부', 'DAS차수', 'DAS지시번호', 'BOXNO',
           'CELLNO', '추가박스여부']
dependent_key = ['플랜트', '상품코드']

In [ ]:
class Product:
    def __init__(self):
        self.products = self.__random_product_creator__()

    def __random_product_creator__(self):
        df = pd.DataFrame({
            '상품코드': df["상품코드"],
            '상품명': df['상품명'],
            'ABC': [['A', 'B', 'C'][np.random.randint(0, 3)] for i in range(len(df))],
            '단품가로(mm)': [np.random.randint(1, 100) for i in range(len(df))],
            '단품세로(mm)': [np.random.randint(1, 100) for i in range(len(df))],
            '단품높이(mm)': [np.random.randint(1, 100) for i in range(len(df))],
            '중량(g)': [np.random.randint(1, 300) for i in range(len(df))]
        })
        return df

    def get_product(self):
        return self.products

In [49]:
class Order:
    def __init__(self, row=1000):
        self.order = pd.DataFrame([], columns=columns)
        self.dependent_set = {}
        self.independent_set = {}
        self.__generator_random_data__()

    def __generator_random_data__(self):
        for col in columns:
            try:
                f = open('Sample/' + col, 'r', encoding='utf-8')
            except:
                f = open('Sample/' + col, 'r', encoding='cp949')
            data = f.read()[:-1].split('|')
            self.independent_set[col] = data

        for i in range(1, 3):
            try:
                f = open('Sample/relation' + str(i), 'r', encoding='utf-8')
            except:
                f = open('Sample/relation' + str(i), 'r', encoding='cp949')
            data = f.read()[:-1].split('\n')
            data = [x.split('|') for x in data]
            self.dependent_set[dependent_key[i - 1]] = data

            for _ in range(row):
                new_random_data = []
                for col in dependent_key:
                    new_random_data.extend(self.__choice_dependent(col))

                for col in columns[6:]:
                    new_random_data.append(self.__choice_independent(col))
                self.order.loc[len(self.order)] = new_random_data

    def __choice_independent__(self, name):
        if name == '주문취소여부' or name == '추가박스여부':
            return random.choices(['Y', 'N'], weights=(5, 95))[0]
        elif name == '운송장출력여부':
            return random.choices(['Y', 'N'], weights=(95, 5))[0]
        else:
            return random.choices(self.Independent_set[name])[0]

    #dependent_set 에서 choice하는 함수.(private)
    #INPUT      - (str) column.
    #OUTPUT     - (str) random choice 되어진 value.
    def __choice_dependent(self, name):
        return random.choices(self.Dependent_set[name])[0]

    #주문 내역 출력함수.
    #INPUT      - none.
    #OUTPUT     - (판다스 DataFrame) 주문내역.
    def show(self):
        return self.order

    #주문 내역 excel 변환 함수.
    #INPUT      - (str) excel 이름.
    def to_excel(self, name='주문 정보'):
        self.order.to_excel(name + '.xlsx', encoding='cp949')  #OUTPUT     - (.xlsx) excel 파일.
        print(f'주문 정보 파일 --- {name}.xlsx 이(가) 생성 되었습니다.')

In [51]:
class SHELF:
    def __init__(self):
        self.products = []

    # 선반에 상품을 적재함
    def push(self, product):
        self.products.append(product)

    # 선반에 상품을 추출함 (선반에 해당 상품이 2개 이상 있는 경우 가장 먼저 들어온 상품을 추출함)
    def pop(self, product):
        for i in range(len(self.products)):
            if self.products[i].code == product.code:
                del self.products[i]
                return

    # 선반에 물건이 몇개 들어있는 지 알려준다.
    def stuffed(self):
        return len(self.products)

    #데이터 보여주기용 함수
    def show(self):
        if len(self.products) == 0:
            print("EMPTY")
        for product in self.products:
            print(product)
        print()


class PRODUCT:
    #  현재 단계에서 잘모르는 데이터에 대해서는 defalut 값으로 처리함
    def __init__(self, code, name='알수없음', ABC='.', x='.', y='.', z='.', g='.'):
        self.code = code
        self.name = name
        self.ABC = ABC
        self.x = x
        self.y = y
        self.z = z
        self.g = g

    def __repr__(self):
        return f"code = {self.code}, name = {self.name}"


class MAP:
    # 워크스테이션 당 맡게 되는 선반의 개수를 일관되게 유지하기 위해서
    # 행, 열 이 r,c 으로 주어진 경우 공장 크기는 r*9, c*7 으로 생성됨

    def __init__(self, R=1, C=1, K=987654321):
        if R < 1 or C < 1:
            raise ValueError("공장 크기가 적절하지 않습니다.\n")
        R *= 9
        C *= 7
        self.R = R
        self.C = C
        self.K = K
        self.index = 1
        self.map = [[0] * C for _ in range(R)]
        self.products = {}

        # 워크스테이션 설정
        ws_index = -1
        for r in range(1, self.R, 9):
            for c in range(3, self.C, 7):
                self.map[r][c] = ws_index
                ws_index -= 1

        # 선반 설정
        break_point = False
        for r in range(3, self.R, 9):
            if break_point: break
            for c in range(1, self.C, 7):
                if break_point: break
                for c1 in range(0, 4, 3):
                    if break_point: break
                    tc = c + c1
                    for dr in range(5):
                        if break_point: break
                        for dc in range(2):
                            cr = r + dr
                            cc = tc + dc
                            self.map[cr][cc] = self.index
                            self.index += 1
                            if self.index > K:
                                break_point = True
                                break

        # 선반 배열 생성
        self.shelves = [SHELF() for _ in range(self.index)]

    # 해당 인덱스를 가진 선반에 물건을 넣는다.
    def push(self, product, index=0, show=False):
        # 인덱스가 변수로 주어져 있지 않다면 비어있는 선반을 찾아서 적재한다.
        if index == 0:
            Full = True
            for i in range(1, self.index):
                if self.shelves[i].stuffed() < 60:
                    index = i
                    Full = False
                    break
            if Full:
                print("모든 선반에 물건이 차있기 때문에 적재할 수 없습니다.\n")
                return

        # 해당 선반에 물건이 60개 이상이라면 적재하지 않는다.
        if self.shelves[index].stuffed() >= 60:
            print("f{index} 번 선반이 가득 차 적재할 수 없습니다.")
            return

        # 해당 선반에 물품을 적재한다. 
        self.shelves[index].push(product)
        # 적재된 상품이 몇번째 선반에 있는지 저장한다.
        if product.code in self.products.keys():
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1
        else:
            self.products[product.code] = [0 for _ in range(self.index)]
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1

        if show:
            print(f"{product.name} 이 {index} 번째 선반에 적재되었습니다.\n")

    # 해당 인덱스를 가진 선반에서 물건을 뺀다.
    def pop(self, product, index=0, show=False):
        # 상품이 공장에 존재하지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return

        # 상품이 공장에 존재하나 인덱스가 주어지지 않은 경우 자동으로 인덱스를 구함
        if index == 0:
            for i, cnt in enumerate(self.products[product.code][1:]):
                if cnt > 0:
                    index = i + 1
                    break

        # 상품이 공장에 존재하고 인덱스가 주어진 경우인데도 해당 선반에 상품이 존재하지 않은 경우
        if self.products[product.code][index] == 0:
            print(f"{product.name} 이 해당 선반에 존재하지 않습니다.")
            return

        # 상품을 추출함
        if show:
            print(f"추출 선반 : {index}, 이동 워크스테이션 : {-((index - 1) // 20) - 1}")
        self.products[product.code][index] -= 1
        self.products[product.code][0] -= 1
        if self.products[product.code][0] == 0:
            del self.products[product.code]
        self.shelves[index].pop(product)

    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 불러와 맵에 적재해 줌
    def push_orders(self, name, head=0, show=False):
        name += '.csv'
        order_data = pd.read_csv(name, encoding='cp949', engine='python')
        if head > 0:
            order_data = order_data[:head]
        columns = ['상품코드', '상품명', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)', '중량(g)']
        for i in range(len(order_data)):
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]],
                              order_data.loc[i][columns[2]],
                              order_data.loc[i][columns[3]], order_data.loc[i][columns[4]],
                              order_data.loc[i][columns[5]],
                              order_data.loc[i][columns[6]])
            self.push(product, show=show)

    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 풀러와 맵에서 추출해 줌
    def pop_orders(self, name, head=0, show=False):
        name += '.csv'
        order_data = pd.read_csv(name, encoding='cp949', engine='python')
        if head > 0:
            order_data = order_data[:head]
        columns = ['상품코드', '상품명', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)', '중량(g)']
        for i in range(len(order_data)):
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]],
                              order_data.loc[i][columns[2]],
                              order_data.loc[i][columns[3]], order_data.loc[i][columns[4]],
                              order_data.loc[i][columns[5]],
                              order_data.loc[i][columns[6]])
            self.pop(product, show=show)

    # 상품이 몇번 선반에 들어있는지 알려준다.
    def show_product(self, product):
        # 상품이 공장에 적재되어있지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return

        print(product, f", 총 {self.products[product.code][0]} 개")
        for index, cnt in enumerate(self.products[product.code][1:]):
            if cnt == 0:
                continue
            print(f"{index + 1}번 선반 - {cnt} 개")
        print()

    # 해당 인덱스를 가진 선반이 어떤 물건을 가지고 있는지 보여준다.
    def show_shelf(self, index):
        print(f"{index} 번째 선반에 들어있는 상품 : ")
        self.shelves[index].show()

    # MAP의 형태를 보여준다. W = workstation, @ = 선반
    # index = True 일 때는 선반의 인덱스를 보여준다. 1,2,3... = 선반의 index, -1 = workstation
    def show(self, index=False):
        print(f"R : {self.R} C : {self.C}\n")

        if index:
            for r in range(self.R):
                for c in range(self.C):
                    print(str(self.map[r][c]).ljust(len(str(self.index))), end=' ')
                print()
            return
        for r in range(self.R):
            for c in range(self.C):
                if self.map[r][c] < 0:
                    print('W', end=' ')
                elif self.map[r][c] > 0:
                    print("@", end=' ')
                else:
                    print('-', end=' ')
            print("\n")


# 사용예시

# 주문 정보 생성

In [63]:
# 주문 정보를 담고 있는 클래스를 생성한다.
random_order = ORDER(2000)
# 클래스가 담고 있는 주문 정보를 엑셀로 만든다.
random_order.to_excel('주문 정보')

주문 정보 파일 --- 주문 정보.xlsx 이(가) 생성 되었습니다.


# 상품 정보 생성

In [64]:
# 주문 정보로 부터 상품 정보를 생성한다.
# 상품 정보가 저장된 'csv' 파일을 만든다.
random_product_creator(in_name='주문 정보', out_name='상품 정보')

상품 정보 파일 --- 상품 정보.csv 이(가) 생성 되었습니다.


# 맵 생성

In [65]:
# 맵 생성
factory = MAP(1, 2)

In [66]:
# 맵 구조 확인
# 선반 = '@', 워크스테이션 = 'W'
factory.show()

R : 9 C : 14

- - - - - - - - - - - - - - 

- - - W - - - - - - W - - - 

- - - - - - - - - - - - - - 

- @ @ - @ @ - - @ @ - @ @ - 

- @ @ - @ @ - - @ @ - @ @ - 

- @ @ - @ @ - - @ @ - @ @ - 

- @ @ - @ @ - - @ @ - @ @ - 

- @ @ - @ @ - - @ @ - @ @ - 

- - - - - - - - - - - - - - 



In [67]:
#맵 구조 확인
# 1 이상의 자연수 : 선반의 인덱스, -1 = 워크스테이션
factory.show(index=True)

R : 9 C : 14

0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  -1 0  0  0  0  0  0  -2 0  0  0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  1  2  0  11 12 0  0  21 22 0  31 32 0  
0  3  4  0  13 14 0  0  23 24 0  33 34 0  
0  5  6  0  15 16 0  0  25 26 0  35 36 0  
0  7  8  0  17 18 0  0  27 28 0  37 38 0  
0  9  10 0  19 20 0  0  29 30 0  39 40 0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  


In [68]:
# 상품 적재
# 상품을 구분하는 기준은 오직 "상품 코드" 하나이다.
# 따라서 상품코드만 다르고 나머지 특성은 전부 같더라도 다른 상품으로 구분한다.
# 반대로 상품코드만 정확히 알고 있다면 상품을 적재, 추출할 수 있다.
factory.push_orders('상품 정보', show=True)

QUEEN BLACK 이 1 번째 선반에 적재되었습니다.

Ballon Half Zip-up (L/grey)(사이즈=L) 이 1 번째 선반에 적재되었습니다.

더퓨어로터스 제주 보태니컬 탈모 샴푸 이 1 번째 선반에 적재되었습니다.

꽃잠 네이처 프리미엄 버진에센스 이 1 번째 선반에 적재되었습니다.

폭스폼폼 이 1 번째 선반에 적재되었습니다.

레이스 업 플레인 스니커즈 BLACK 225 이 1 번째 선반에 적재되었습니다.

제마 폰월렛 SKY BLUE XXX 이 1 번째 선반에 적재되었습니다.

베르벨라커버아트파운데이션팩트포커버레벨4,피치,,20,G 이 1 번째 선반에 적재되었습니다.

[TA221VT53P] stitch slim vest-ivory-2 이 1 번째 선반에 적재되었습니다.

테일러 미니미 팝 크로스바디 RASPBERRY XXX 이 1 번째 선반에 적재되었습니다.

모르간 크로스바디 CARAMEL XXX 이 1 번째 선반에 적재되었습니다.

웨이브백 세로형 베이지 이 1 번째 선반에 적재되었습니다.

마들렌 버킷 BLACK XXX 이 1 번째 선반에 적재되었습니다.

공동체카드 스티커 이 1 번째 선반에 적재되었습니다.

동물마을 쥐돌이 이 1 번째 선반에 적재되었습니다.

나디아 숄더 BLACK XXX 이 1 번째 선반에 적재되었습니다.

그리니 하이탑 스니커즈 BLACK 이 1 번째 선반에 적재되었습니다.

루키 크랙 크로스바디-BUTTER CREAM 이 1 번째 선반에 적재되었습니다.

치매를 위한 작업치료 중재 용어집 이 1 번째 선반에 적재되었습니다.

에어퀸 네오마스크(2매)(라이트 코랄)[낱장피킹] 이 1 번째 선반에 적재되었습니다.

SPANGLE POINT T-SHIRT NAVY 0FR 이 1 번째 선반에 적재되었습니다.

SUEDE LONG TRENCH COAT NAVY 066 이 1 번째 선반에 적재되었습니다.

KL530BXI, (50)Blue, 120 이 1 번째 선반에 적재되었습니다.

에어퀸 에센셜 생리대 슈퍼롱 

In [72]:
# 적재된 상품 확인
# 1번 선반에 적재된 상품 확인
factory.show_shelf(4)

4 번째 선반에 들어있는 상품 : 
code = DEF003, name = 블랙 삼각 덤벨(3kg)
code = s0019, name = 리밸런스비니거 1L병
code = 0081010, name = PET BED-Grey 385
code = s0023, name = 마이 애사비 라인 핏 2개월
code = O1FBBC27139XXX, name = 카일리 버킷 FOSSIL TAUPE
code = O1SBCB68004XXX, name = 세라클 크로스바디 CREAM
code = O2SPLP16009XXX, name = 맨디 2단 장지갑+BLACK
code = 9BDS0000210, name = 그란디디에 다즐링홍차 설탕클렌징세럼 150ML
code = 8801121768075, name = [앨리슨]아몬드 브리즈 바나나 190ml 24개입
code = O0FBCB43009XXX, name = 루키크로스바디 BLACK
code = O1SBCB01009XXX, name = 테일러 글래드 크로스바디 BLACK XXX
code = 900015, name = 워터페인트 250ml 초록색
code = roll, name = 스펀지 롤러
code = O1SBBC02009XXX, name = 테일러버킷 BLACK
code = 8809738603512, name = 드 메모리아 #04 드레스덴
code = O2SPCC07157XXX, name = 루시 카드케이스+SUN GREEN
code = IN-AK-005, name = 인소의 법칙 아크릴 키링 E. 우주인
code = O9SBCB81127XXX, name = 로티새들크로스바디 TRUE NAVY XXX
code = BGLMUMA001-V01, name = 벨그림 리치컬링 픽스카라
code = O2SBSD14009XXX, name = 아미 숄더_BLACK
code = O0SCTS080010FR, name = HEART EMBROIDERY T-SHIRTS O0SCTS080010FR
code = NKPM6S541G0M13000, 

In [73]:
# 적재된 상품 확인
# 꺼내줘양이 적재된 선반의 위치 확인
factory.show_product(PRODUCT('CMZ111257', '꺼내줘양'))

code = CMZ111257, name = 꺼내줘양 , 총 2 개
4번 선반 - 1 개
30번 선반 - 1 개



In [75]:
# 상품 추출 : 1개
factory.pop(PRODUCT('CMZ111257'), index=4, show=True)

추출 선반 : 4, 이동 워크스테이션 : -1


In [76]:
factory.pop(PRODUCT('CMZ111257'), index=30, show=True)

추출 선반 : 30, 이동 워크스테이션 : -2


In [78]:
# 상품 추출 : 2개 이상
factory.pop_orders('상품 정보', show=True)

추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이동 워크스테이션 : -1
추출 선반 : 1, 이

In [79]:
# 상품이 공장에서 추출되었는지 확인
factory.show_shelf(1)

1 번째 선반에 들어있는 상품 : 
EMPTY



In [80]:
# 꺼내줘양이 추출되었는지 확인
factory.show_product(PRODUCT('CMZ111257', '꺼내줘양'))

꺼내줘양 이 공장에 존재하지 않습니다.

